# Gold Data Load

In this notebook, the Silver tables will be combined into a single dataframe. And then modified in several ways: 

Players with fewer than 100 (or a value set by the user) total games played will be omitted as we are attempting to track year over year performance for career NHL players.  Players who do not play more than 2 complete seasons would likely be unhelpful.  Additionally, players who have appeared in the NHL in fewer than 3 (also adjustable by the user) seasons will be excluded.  Stints shorter than 10 games will be excluded as well, as these are small sample sizes and the players performance during that brief period could be high or low due to normal variance.

Aggregated data will be found for each player.  This data will include items like "total games played", "goals per game per stint", "number of same birth country teammates per playerID & year combination", "number of years played in NHL".

Then, each player's stats will be normalized so that their worst stint in a given stat is recorded as a 0, the best stint is a 1, and all other stints fall in between.  These normalized stats will then be used to calculate an overall score for that player's stint will be calculated by adding all of these scores together (with optional weights to increase or reduce the impact of certain stats). We will then be able to compare the overall score for that stint with the number of teammates with the same birth country to study the correlation between these two items.

<hr>

# Gold Process
<hr>

## Handle Missing Values

In the NHL's history, several stats were not tracked until later.  These include: +/-, PPG. PPA, SHG, SHA, GWG, SOG. For players who have records in the time period where these stats where both tracked and untracked, it would be difficult to compare their performance between years where data for these stats is and is not present. For this reason, these stats will not be considered for players where their earliest year has a null value for +/-. If +/- is null in their earliest year, all of that player's stats from the aforementioned list will be updated to 0 values.

Additionally, any missing value in any of the numeric stat columns is replaced with a 0.

In [22]:
import pandas as pd

# load table
df = pd.read_csv("Data/Silver/Scoring.csv")

# Define the stats to be checked and updated
stats = ['+/-', 'PPG', 'PPA', 'SHG', 'SHA', 'GWG', 'SOG']

# Sort the dataframe by playerID and year
df = df.sort_values(by=['playerID', 'year'])

print(df[stats].info())

# Identify players whose earliest year has null for any of the specified stats
players_with_null_stats = df.groupby('playerID').first()[stats].isnull().any(axis=1)
players_with_null_stats = players_with_null_stats[players_with_null_stats].index

# Update all records for those players to 0 for the specified stats
df.loc[df['playerID'].isin(players_with_null_stats), stats] = 0



print(df[stats].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38153 entries, 0 to 38152
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   +/-     31506 non-null  float64
 1   PPG     32154 non-null  float64
 2   PPA     21101 non-null  float64
 3   SHG     32154 non-null  float64
 4   SHA     21275 non-null  float64
 5   GWG     32199 non-null  float64
 6   SOG     31507 non-null  float64
dtypes: float64(7)
memory usage: 2.0 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38153 entries, 0 to 38152
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   +/-     38153 non-null  float64
 1   PPG     38153 non-null  float64
 2   PPA     34974 non-null  float64
 3   SHG     38153 non-null  float64
 4   SHA     35143 non-null  float64
 5   GWG     38153 non-null  float64
 6   SOG     38153 non-null  float64
dtypes: float64(7)
memory usage: 2.0 MB
None



## Build Wide Table

### Load Silver Scoring

We will start by loading the Scoring table as it is the main fact table we will be using. We will then drop any rows with 0 or NaN GP.  There are a few rows where the stint has 0 games played. These should not be included.

In [2]:
# Remove NaN GP rows
df = df.dropna(subset=['GP'])


### Drop Players With Too Few Total GP Or Years

In [3]:
# calculate total GP
total_gp_per_player = df.groupby('playerID')['GP'].sum()

# identify players with too few games
game_threshold = 100
players_to_remove = total_gp_per_player[total_gp_per_player < game_threshold].index

# Remove rows from df with these players
df = df[~df['playerID'].isin(players_to_remove)]

### Calculate Longest Stint

Next we will determine which stint each year had the most games played for that player.  This will be used to attribute Awards to that player.

In [4]:
# Identify which stint has the most games played per player per year
df['longest_stint'] = 0

# Group by playerID and year, and get the index of the max GP within each group
idx = df.groupby(['playerID', 'year'])['GP'].idxmax()

# Update the 'longest_stint' column to 1 for the rows with the max GP within each group
df.loc[idx, 'longest_stint'] = 1

### Joining Awards

Awards for each player will be attributed to the stint where the player played the most games that season. Awards will be joined to Scoring on playerID and year where longest_stint = 1.  Instead of joining actual award information, we will just load the number of awards earned by the player that year.

In [5]:
import pandas as pd

# Load the main dataframe (assuming it is already loaded as df)
# df = pd.read_csv('Data/Gold/main_table.csv')  # Uncomment if you need to load the main dataframe

# Load Awards table
df_awards = pd.read_csv('Data/Silver/Awards.csv')

# Find award count per player per year
df_awards = df_awards.groupby(['playerID', 'year']).size().reset_index(name='award_count')

# Filter the main dataframe to include only the longest stint
filtered_df = df[df['longest_stint'] == 1]

# Merge the filtered df with the aggregated awards table on playerID and year
merged_df = pd.merge(filtered_df, df_awards, on=['playerID', 'year'], how='left')

# Fill NaN values in award_count with 0 (assuming players without awards should have a count of 0)
merged_df['award_count'] = merged_df['award_count'].fillna(0)

# Assign the merged dataframe back to the original dataframe to include the new column
df.loc[filtered_df.index, 'award_count'] = merged_df['award_count']

# Verify the resulting DataFrame
print(df.head())

    playerID  year  stint tmID pos    GP    G     A   Pts   PIM   +/-  PPG  \
0  aaltoan01  1997      1  ANA   C   3.0  0.0   0.0   0.0   0.0  -1.0  0.0   
1  aaltoan01  1998      1  ANA   C  73.0  3.0   5.0   8.0  24.0 -12.0  2.0   
2  aaltoan01  1999      1  ANA   C  63.0  7.0  11.0  18.0  26.0 -13.0  1.0   
3  aaltoan01  2000      1  ANA   C  12.0  1.0   1.0   2.0   2.0   1.0  0.0   
5  abdelju01  2007      1  DET   L   2.0  0.0   0.0   0.0   2.0   0.0  0.0   

   PPA  SHG  SHA  GWG    SOG  longest_stint  award_count  
0  0.0  0.0  0.0  0.0    1.0              1          0.0  
1  1.0  0.0  0.0  0.0   61.0              1          0.0  
2  0.0  0.0  0.0  1.0  102.0              1          0.0  
3  0.0  0.0  0.0  0.0   18.0              1          0.0  
5  0.0  0.0  0.0  0.0    6.0              1          0.0  


### Joining Teams

Teams will join on tmID and year in both Teams and Scoring. From Teams, we will add the name column to Scoring as team_name.

In [6]:
# Load Teams table
teams_df = pd.read_csv('Data/Silver/Teams.csv')

# Rename the 'name' column to 'team_name'
teams_df.rename(columns={'name': 'team_name'}, inplace=True)

# Merge the main dataframe with the Teams dataframe on 'year' and 'tmID'
merged_df = pd.merge(df, teams_df, on=['year', 'tmID'], how='left')

# Print the first few rows of the merged dataframe to verify the merge
print("Merged DataFrame:\n", merged_df.head())

Merged DataFrame:
     playerID  year  stint tmID pos    GP    G     A   Pts   PIM   +/-  PPG  \
0  aaltoan01  1997      1  ANA   C   3.0  0.0   0.0   0.0   0.0  -1.0  0.0   
1  aaltoan01  1998      1  ANA   C  73.0  3.0   5.0   8.0  24.0 -12.0  2.0   
2  aaltoan01  1999      1  ANA   C  63.0  7.0  11.0  18.0  26.0 -13.0  1.0   
3  aaltoan01  2000      1  ANA   C  12.0  1.0   1.0   2.0   2.0   1.0  0.0   
4  abdelju01  2007      1  DET   L   2.0  0.0   0.0   0.0   2.0   0.0  0.0   

   PPA  SHG  SHA  GWG    SOG  longest_stint  award_count  \
0  0.0  0.0  0.0  0.0    1.0              1          0.0   
1  1.0  0.0  0.0  0.0   61.0              1          0.0   
2  0.0  0.0  0.0  1.0  102.0              1          0.0   
3  0.0  0.0  0.0  0.0   18.0              1          0.0   
4  0.0  0.0  0.0  0.0    6.0              1          0.0   

                 team_name  
0  Mighty Ducks of Anaheim  
1  Mighty Ducks of Anaheim  
2  Mighty Ducks of Anaheim  
3  Mighty Ducks of Anaheim  
4     

### Joining Skaters

In this step, we will add much of the biographical data kept in Skaters into the Scoring table. This will include birth country, height, weight, and others.

In [7]:
# Load Skaters table
skaters_df = pd.read_csv('Data/Silver/Skaters.csv')

# Print the first few rows of the skaters dataframe to verify its contents
print("Skaters DataFrame:\n", skaters_df.head())

# Merge the main dataframe with the Skaters dataframe on 'playerID'
df = pd.merge(df, skaters_df, on='playerID', how='left')

# Verify the resulting DataFrame
print("Merged DataFrame:\n", df.head())

Skaters DataFrame:
     playerID firstName    lastName  height  weight  birthYear birthCountry
0  aaltoan01     Antti       Aalto    73.0   210.0     1975.0      Finland
1  abbeybr01     Bruce       Abbey    73.0   185.0     1951.0       Canada
2  abbotre01       Reg      Abbott    71.0   164.0     1930.0       Canada
3  abdelju01    Justin  Abdelkader    73.0   195.0     1987.0          USA
4   abelcl01  Clarence        Abel    73.0   225.0     1900.0          USA
Merged DataFrame:
     playerID  year  stint tmID pos    GP    G     A   Pts   PIM  ...  GWG  \
0  aaltoan01  1997      1  ANA   C   3.0  0.0   0.0   0.0   0.0  ...  0.0   
1  aaltoan01  1998      1  ANA   C  73.0  3.0   5.0   8.0  24.0  ...  0.0   
2  aaltoan01  1999      1  ANA   C  63.0  7.0  11.0  18.0  26.0  ...  1.0   
3  aaltoan01  2000      1  ANA   C  12.0  1.0   1.0   2.0   2.0  ...  0.0   
4  abdelju01  2007      1  DET   L   2.0  0.0   0.0   0.0   2.0  ...  0.0   

     SOG  longest_stint  award_count  firstName 

## Calculated Fields

In this section, the calculated fields that will be used to evaluate player performance will be introduced.

### Per-Game Averages

In order to compare stints with each other, they need to be normalized by the number of games played.

In [8]:
# List of columns to convert to per-game averages
stats_columns = ['G', 'A', 'Pts', 'PIM', 'PPG', 'PPA', 'SHG', 'SHA', 'GWG', 'SOG', 'award_count']
per_game_columns = list(map(lambda col: col + '_per_game', stats_columns))
per_game_columns.append('+/-')

# Calculate per-game averages and add them as new columns
for col in stats_columns:
    df[col + '_per_game'] = df[col] / df['GP']

df.head()

,playerID,year,stint,tmID,pos,GP,G,A,Pts,PIM,...,A_per_game,Pts_per_game,PIM_per_game,PPG_per_game,PPA_per_game,SHG_per_game,SHA_per_game,GWG_per_game,SOG_per_game,award_count_per_game
0,aaltoan01,1997,1,ANA,C,3.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.333333,0.0
1,aaltoan01,1998,1,ANA,C,73.0,3.0,5.0,8.0,24.0,...,0.068493,0.109589,0.328767,0.027397,0.013699,0.0,0.0,0.000000,0.835616,0.0
2,aaltoan01,1999,1,ANA,C,63.0,7.0,11.0,18.0,26.0,...,0.174603,0.285714,0.412698,0.015873,0.000000,0.0,0.0,0.015873,1.619048,0.0
3,aaltoan01,2000,1,ANA,C,12.0,1.0,1.0,2.0,2.0,...,0.083333,0.166667,0.166667,0.000000,0.000000,0.0,0.0,0.000000,1.500000,0.0
4,abdelju01,2007,1,DET,L,2.0,0.0,0.0,0.0,2.0,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.000000,3.000000,0.0


## Normalize Per-Game Averages

In this step, each player's per-game averages will be normalized across their career such that their worst year in a particular stat will have a score of 0, their best year will have a score of 1, and all other years will have a score between 0 and 1, relative to that year's performance.

In [9]:
# Function to normalize a series with an epsilon
def normalize_series_with_epsilon(series, epsilon=1e-6):
    min_val = series.min()
    max_val = series.max()
    if min_val == max_val:
        return series.apply(lambda x: 0.5)  # If all values are the same, return a middle value
    return (series - min_val) / (max_val - min_val + epsilon)

# Group by playerID and normalize per-game columns
df_normalized = df.copy()
for col in per_game_columns:
    df_normalized[col + '_score'] = df.groupby('playerID')[col].transform(lambda x: normalize_series_with_epsilon(x))

# Display the updated dataframe
print(df_normalized)

# update df
df = df_normalized

        playerID  year  stint tmID pos    GP    G     A   Pts   PIM  ...  \
0      aaltoan01  1997      1  ANA   C   3.0  0.0   0.0   0.0   0.0  ...   
1      aaltoan01  1998      1  ANA   C  73.0  3.0   5.0   8.0  24.0  ...   
2      aaltoan01  1999      1  ANA   C  63.0  7.0  11.0  18.0  26.0  ...   
3      aaltoan01  2000      1  ANA   C  12.0  1.0   1.0   2.0   2.0  ...   
4      abdelju01  2007      1  DET   L   2.0  0.0   0.0   0.0   2.0  ...   
...          ...   ...    ...  ...  ..   ...  ...   ...   ...   ...  ...   
31932  zyuzian01  2002      2  MIN   D  66.0  4.0  12.0  16.0  34.0  ...   
31933  zyuzian01  2003      1  MIN   D  65.0  8.0  13.0  21.0  48.0  ...   
31934  zyuzian01  2005      1  MIN   D  57.0  7.0  11.0  18.0  50.0  ...   
31935  zyuzian01  2006      1  CAL   D  49.0  1.0   5.0   6.0  30.0  ...   
31936  zyuzian01  2007      1  CHI   D  32.0  2.0   3.0   5.0  38.0  ...   

       Pts_per_game_score  PIM_per_game_score  PPG_per_game_score  \
0                0

## Calculate Overall Score for Stint

In this step, all score columns are added together to form a score for the player. Optional weights are included as well.  All weights will be coefficients to the different score columns when calculating the overall score.

In [10]:
# Define Weights
G_weight = 1
A_weight = 1
Pts_weight = 1
PIM_weight = 1
plus_minus_weight = 1  # Converted +/- to plus_minus
PPG_weight = 1
PPA_weight = 1
SHG_weight = 1
SHA_weight = 1
GWG_weight = 1
SOG_weight = 1
award_count_weight = 1

# Calculate stint_score
df['stint_score'] = (
    df['G_per_game_score'] * G_weight +
    df['A_per_game_score'] * A_weight +
    df['Pts_per_game_score'] * Pts_weight +
    df['PIM_per_game_score'] * PIM_weight +
    df['PPG_per_game_score'] * PPG_weight +
    df['PPA_per_game_score'] * PPA_weight +
    df['SHG_per_game_score'] * SHG_weight +
    df['SHA_per_game_score'] * SHA_weight +
    df['GWG_per_game_score'] * GWG_weight +
    df['SOG_per_game_score'] * SOG_weight +
    df['+/-_score'] * plus_minus_weight +
    df['award_count_per_game_score'] * award_count_weight
)

## Compare the Stint to the Previous Stint - overall_score CURRENTLY INCOMPLETE

In this step, a column is added that adds an index to the player per stint. Then, a new column is added that checks if the overall score of the current stint is higher or lower than the previous stint. This column will hold the value "higher" or "lower".



In [11]:
df['overall_score_vs_prev_stint'] = pd.NA

## Number of Teammates with Same Nationality - CURRENTLY INCOMPLETE

In [12]:
df['teammates_same_nationality'] = pd.NA

## Compare the Stint to the Previous Stint - teammates_same_nationality

In [13]:
df['tsm_vs_prev_stint'] = pd.NA

In [14]:
file_path ='Data/Gold/main.csv'

df.to_csv(f'{file_path}', index=False)

import os

file_size = os.path.getsize(file_path)

# Convert the file size to a more readable format (e.g., kilobytes or megabytes)
file_size_kb = file_size / 1024
file_size_mb = file_size_kb / 1024

# Display the file size
print(f'=============main=================')
print(f'The file size is {file_size} bytes')
print(f'The file size is {file_size_kb:.2f} KB')
print(f'The file size is {file_size_mb:.2f} MB')
print('===================================')
print(f'{df.shape}')

=============main=================
The file size is 11483119 bytes
The file size is 11213.98 KB
The file size is 10.95 MB
(31937, 52)
